In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/bigmart-sales-data/Train.csv')
df.head(10)

In [ ]:
df.columns

In [ ]:
df.dtypes
# fetures having "object" dtype
# ['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Size,'Outlet_Location_Type','Outlet_Type']

# remaining all are of float or int dtype

In [ ]:
df['Item_Fat_Content'].unique()

In [ ]:
df['Item_Fat_Content'].value_counts()


# Low Fat    5089
# Regular    2889
# LF          316
# reg         117
# low fat     112

In [ ]:
df['Item_Type'].unique()

In [ ]:
df['Item_Type'].value_counts()

In [ ]:
df['Outlet_Identifier'].value_counts()

In [ ]:
df['Outlet_Size'].value_counts()

In [ ]:
df['Outlet_Location_Type'].value_counts()

In [ ]:
df['Outlet_Type'].value_counts()

In [ ]:
df.isna().sum()

In [ ]:
df.shape

# **EDA(Exploratory Data Analysis)**

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.head(2)

In [ ]:
f,axes=plt.subplots(1,2,figsize=[14,6])
sns.distplot(df['Item_MRP'],ax=axes[0])
sns.violinplot(df['Item_MRP'],ax=axes[1])


In [ ]:
f,axes=plt.subplots(1,2,figsize=[14,6])
sns.distplot(df['Item_Visibility'],ax=axes[0])
sns.violinplot(df['Item_Visibility'],ax=axes[1])

In [ ]:
f,axes=plt.subplots(1,2,figsize=[14,7])
sns.countplot('Item_Fat_Content',data=df,ax=axes[0])
chart=sns.countplot('Item_Type',data=df,ax=axes[1])
chart.set_xticklabels(chart.get_xticklabels(), rotation=45,horizontalalignment='right')

In [ ]:
f,axes=plt.subplots(1,2,figsize=[15,6])
chart=sns.countplot('Outlet_Establishment_Year',data=df,ax=axes[1])
chart.set_xticklabels(chart.get_xticklabels(), rotation=45,horizontalalignment='right')

sns.countplot('Outlet_Location_Type',data=df,ax=axes[0])


In [ ]:
 df['Outlet_Location_Type'] 

In [ ]:
df.drop(['Item_Weight','Outlet_Size'],axis=1,inplace=True)

In [ ]:
df=pd.get_dummies(df,columns=['Item_Fat_Content','Item_Type','Outlet_Location_Type','Outlet_Type'],drop_first=True)

In [ ]:
df.corr()

In [ ]:
df

drop 'Item_Fat_Content_low fat' and 'Item_Fat_Content_reg' because their value is very less 

In [ ]:
df.drop(['Item_Fat_Content_low fat','Item_Fat_Content_reg'],axis=1,inplace=True)

In [ ]:
df.columns

drop these columns due to less contribution 

In [ ]:
df.drop([ 'Item_Type_Breads', 'Item_Type_Breakfast', 'Item_Type_Canned',
       'Item_Type_Dairy',
        'Item_Type_Hard Drinks',
       'Item_Type_Health and Hygiene', 'Item_Type_Meat',
       'Item_Type_Others', 'Item_Type_Seafood', 
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods',],axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df.corr(),annot=True)

drop these columns due to negligible corelation with target value

In [ ]:
df.drop(['Outlet_Type_Supermarket Type2','Item_Type_Frozen Foods','Item_Fat_Content_Low Fat','Item_Visibility','Outlet_Establishment_Year','Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.columns

In [ ]:
X=df.drop('Item_Outlet_Sales',axis=1)
X=X.values
X.shape


In [ ]:
y=df['Item_Outlet_Sales']
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [ ]:
X_train.shape,y_train.shape

In [ ]:
X_test.shape,y_test.shape

# Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit_transform(X,y)

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
random_classifier = RandomForestRegressor()
parameters = { 'max_features':np.arange(4,7),'n_estimators':np.arange(50,150,7),'min_samples_leaf': [5,10,15]}
random_grid = GridSearchCV(random_classifier, parameters, cv = 7)

In [ ]:
random_grid.fit(X_train,y_train)

In [ ]:
random_grid.best_params_

In [ ]:
y_pred_test=random_grid.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred_test)

In [ ]:
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_test)))

In [ ]:
y_pred_train=random_grid.predict(X_train)
mean_squared_error(y_train, y_pred_train)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg=LinearRegression()
reg.fit(X_train,y_train)

In [ ]:
y_pred=reg.predict(X_test)
mean_squared_error(y_test, y_pred)

In [ ]:
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))